
<center>
    <p style="text-align:center">
    <img alt="arize logo" src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="300"/>
        <br>
        <a href="https://docs.arize.com/arize/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/client_python">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# <center>Tracing with CrewAI</center>    

This guide demonstrates how to use Arize with CrewAI for tracing. 

You can read more about LLM tracing [here](https://docs.arize.com/arize/llm-large-language-models/llm-traces). In this tutorial, you will use opentelemetry and [openinference](https://github.com/Arize-ai/openinference/tree/main) to instrument our application in order to trace your LLM and agent calls with CrewAI.

ℹ️ This notebook requires:
- A OpenAI API key
- A Serper API key (this is at [serper.dev](https://serper.dev) and has a free tier)
- An Arize Space & API Key


## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies.

In [1]:
# Dependencies needed to instrument your openai application using opentelemetry and openinference
!pip3 install arize-otel openinference-instrumentation-crewai opentelemetry-sdk opentelemetry-exporter-otlp opentelemetry-instrumentation-httpx crewai crewai-tools openinference-instrumentation-litellm


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


## Step 2: Get your API Keys

Copy the Arize API_KEY and SPACE_ID from your Space Settings page (shown below) to the variables in the cell below.

<center><img src="https://storage.googleapis.com/arize-assets/barcelos/Screenshot%202024-11-11%20at%209.28.27%E2%80%AFPM.png" width="700"></center>

In [9]:
import os
from getpass import getpass
from dotenv import load_dotenv

# Load environment variables from .env file if you have one
load_dotenv()

if not (SPACE_ID := os.getenv("SPACE_ID")):
    SPACE_ID = getpass("🔑 Enter your Space ID: ")

if not (ARIZE_API_KEY := os.getenv("ARIZE_API_KEY")):
    ARIZE_API_KEY = getpass("🔑 Enter your Arize API Key: ")

if not (OPENAI_API_KEY := os.getenv("OPENAI_API_KEY")):
    OPENAI_API_KEY = getpass("🔑 Enter your OpenAI API key: ")

if not (SERPER_API_KEY := os.getenv("SERPER_API_KEY")):
    SERPER_API_KEY = getpass("🔑 Enter your Serper API key: ")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

## Step 3. Add our tracing code
We will be using the arize_otel package to register the URL and authentication parameters to send to Arize using OpenTelemetry. You can see what's under the hood by looking [here](https://docs.arize.com/arize/large-language-models/tracing/auto-instrumentation).

In [7]:
# Import open-telemetry dependencies
from arize_otel import register_otel, Endpoints
from openinference.instrumentation.crewai import CrewAIInstrumentor
from openinference.instrumentation.litellm import LiteLLMInstrumentor

# Setup OTEL via our convenience function
register_otel(
    endpoints=Endpoints.ARIZE,
    space_id=SPACE_ID,  # in app space settings page
    api_key=ARIZE_API_KEY,  # in app space settings page
    project_name="crewai-tracing-tutorial",  # name this to whatever you would like
)

CrewAIInstrumentor().instrument()
LiteLLMInstrumentor().instrument()

Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


## Step 4: Run your LLM application
Let's create two agents that work together to write a blog post and see the results in Arize.

In [10]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import SerperDevTool

search_tool = SerperDevTool()

# Define your agents with roles and goals
researcher = Agent(
    role="Senior Research Analyst",
    goal="Uncover cutting-edge developments in AI and data science",
    backstory="""You work at a leading tech think tank.
  Your expertise lies in identifying emerging trends.
  You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    # You can pass an optional llm attribute specifying what model you wanna use.
    # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7),
    tools=[search_tool],
)
writer = Agent(
    role="Tech Content Strategist",
    goal="Craft compelling content on tech advancements",
    backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
  You transform complex concepts into compelling narratives.""",
    verbose=True,
    allow_delegation=True,
)

# Create tasks for your agents
task1 = Task(
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
    expected_output="Full analysis report in bullet points",
    agent=researcher,
)

task2 = Task(
    description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
    expected_output="Full blog post of at least 4 paragraphs",
    agent=writer,
)

# Instantiate your crew with a sequential process
crew = Crew(
    agents=[researcher, writer],
    tasks=[task1, task2],
    verbose=1,
    process=Process.sequential,
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

2024-12-05 16:33:50,302 - 8342091840 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


# Agent: Senior Research Analyst
## Thought: I need to gather information about the latest advancements in AI in 2024, including key trends, breakthrough technologies, and potential industry impacts. To do this, I will search for up-to-date information on AI developments in 2024.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"latest advancements in AI 2024 key trends breakthrough technologies industry impacts\"}"
## Tool Output: 

Search results: Title: 11 New Technologies in AI: All Trends of 2023-2024 - devabit
Link: https://devabit.com/blog/top-11-new-technologies-in-ai-exploring-the-latest-trends/
Snippet: From the remarkable advancements in natural language processing (NLP) and GPT to the impact of AI on the sales and marketing industries, we will explore th

2024-12-05 16:34:02,938 - 8342091840 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Senior Research Analyst
## Final Answer: 
Comprehensive Analysis of the Latest Advancements in AI in 2024**

- **Key Trends:**
  - **Generative AI Proliferation:**
    - Continued growth and democratization of generative AI technologies are making them accessible to a wider audience, allowing non-specialists to leverage AI in creative and productive ways.
  - **Multimodal AI:**
    - AI systems that can process and integrate multiple forms of data (text, image, audio) are becoming increasingly prevalent, enabling richer and more versatile applications.
  - **AI for Workplace Productivity:**
    - Businesses are adopting AI tools for enhanced workplace productivity, including automation of routine tasks, data analysis, and customer interaction.
  - **Agentic AI:**
    - Development of AI systems capable of autonomous decision-making is gaining momentum, leading to more sophisticated AI agents that can undertake complex tasks with minimal human intervention.
  - **Ethical AI a

2024-12-05 16:34:30,388 - 8342091840 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Tech Content Strategist
## Final Answer: 
**Overview of AI Advancements in 2024**

As we move deeper into 2024, the landscape of artificial intelligence continues to evolve rapidly. Several key trends are reshaping industries and influencing how organizations leverage technology. Here’s a comprehensive overview of the latest advancements in AI, including generative AI proliferation, multimodal AI, workplace productivity enhancements, and ethical considerations.

### Key Trends

1. **Generative AI Proliferation**: 
   The growth of generative AI has been exponential in 2024, with tools that can autonomously create text, images, music, and more. Generative AI applications are being utilized in creative industries, education, and content generation, with companies integrating these capabilities into their workflows to enhance productivity and creativity.

2. **Multimodal AI**: 
   Multimodal AI systems, which can process and generate data across various modalities (text, speech

Great! Our application works!

## Step 5: Log into Arize and explore your application traces 🚀

Log into your Arize account, and look for the project with the same `project_name`.

<img src="https://storage.googleapis.com/arize-assets/tutorials/images/arize-llm-tracing.png" width="1500" />